In [21]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [29]:
# Read data from CSV file
data_df = pd.read_csv("Most_Popular_Baby_Names_in_NYC.csv")
data_df.shape

(13962, 6)

In [9]:
# Columns of dataset
data_df.columns

Index(['brth_yr', 'gndr', 'ethcty', 'nm', 'cnt', 'rnk'], dtype='object')

In [10]:
# First five records of dataset
data_df.head()

,brth_yr,gndr,ethcty,nm,cnt,rnk
0,2011,FEMALE,HISPANIC,GERALDINE,13,75
1,2011,FEMALE,HISPANIC,GIA,21,67
2,2011,FEMALE,HISPANIC,GIANNA,49,42
3,2011,FEMALE,HISPANIC,GISELLE,38,51
4,2011,FEMALE,HISPANIC,GRACE,36,53


In [11]:
# Number of unique values for each column of dataset
columns = list(data_df.columns)
for column in columns:
    print(f"len(unique({column})) = {len(pd.unique(data_df[column]))}")

len(unique(brth_yr)) = 4
len(unique(gndr)) = 2
len(unique(ethcty)) = 7
len(unique(nm)) = 2811
len(unique(cnt)) = 258
len(unique(rnk)) = 102


In [13]:
# Unique values for columns whose number of unique values is less than or equal to 10
columns = list(data_df.columns)
for column in columns:
    unique_values = pd.unique(data_df[column])
    if(len(unique_values) <= 10):
        print(f"unique({column}) = {unique_values}")

unique(brth_yr) = [2011 2012 2013 2014]
unique(gndr) = ['FEMALE' 'MALE']
unique(ethcty) = ['HISPANIC' 'WHITE NON HISPANIC' 'ASIAN AND PACIFIC ISLANDER'
 'BLACK NON HISPANIC' 'ASIAN AND PACI' 'BLACK NON HISP' 'WHITE NON HISP']


In [30]:
# Encoding the gndr column into two separate columns
data_df["fml"] = np.select([data_df["gndr"] == "FEMALE"], [1], default=0)
data_df["ml"] = np.select([data_df["gndr"] == "MALE"], [1], default=0)

# drop the gndr column as it's no longer needed
del data_df["gndr"]
data_df.head()

,brth_yr,ethcty,nm,cnt,rnk,fml,ml
0,2011,HISPANIC,GERALDINE,13,75,1,0
1,2011,HISPANIC,GIA,21,67,1,0
2,2011,HISPANIC,GIANNA,49,42,1,0
3,2011,HISPANIC,GISELLE,38,51,1,0
4,2011,HISPANIC,GRACE,36,53,1,0


In [31]:
# Encode the ethcty column into separate columns
data_df["hispanic"] = np.select(
    [data_df["ethcty"] == "HISPANIC"],
    [1],
    default=0
)

data_df["white"] = np.select(
    [(data_df["ethcty"] == "WHITE NON HISPANIC") | (data_df["ethcty"] == "WHITE NON HISP")],
    [1],
    default=0
)

data_df["asian_and_pacific_islander"] = np.select(
    [(data_df["ethcty"] == "ASIAN AND PACIFIC ISLANDER") | (data_df["ethcty"] == "ASIAN AND PACI")],
    [1], 
    default=0
)

data_df["black"] = np.select(
    [(data_df["ethcty"] == "BLACK NON HISPANIC") | (data_df["ethcty"] == "BLACK NON HISP")],
    [1],
    default=0
)

# Drop the ethcy column as it's no longer needed
del data_df["ethcty"]
data_df.head()

,brth_yr,nm,cnt,rnk,fml,ml,hispanic,white,asian_and_pacific_islander,black
0,2011,GERALDINE,13,75,1,0,1,0,0,0
1,2011,GIA,21,67,1,0,1,0,0,0
2,2011,GIANNA,49,42,1,0,1,0,0,0
3,2011,GISELLE,38,51,1,0,1,0,0,0
4,2011,GRACE,36,53,1,0,1,0,0,0


In [32]:
# Drop the name column
del data_df["nm"]
data_df.head()

,brth_yr,cnt,rnk,fml,ml,hispanic,white,asian_and_pacific_islander,black
0,2011,13,75,1,0,1,0,0,0
1,2011,21,67,1,0,1,0,0,0
2,2011,49,42,1,0,1,0,0,0
3,2011,38,51,1,0,1,0,0,0
4,2011,36,53,1,0,1,0,0,0
